In [9]:
import pandas as pd
import dbconfig as dbConf

In [44]:
def GetPatten():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    #print("접속완료")

    sql = ("""
           select stan
                , 3cnt
                , 5cnt
                , 10cnt
                , 30cnt
             from in_list_2
            limit 15
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [124]:
df = GetPatten()

stan  = list(range(0,1))
cnt3  = list(range(0,1))
cnt5  = list(range(0,1))
cnt10 = list(range(0,1))
cnt30 = list(range(0,1))

for df_stan, df_cnt3, df_cnt5, df_cnt10, df_cnt30 in zip(df['stan'], df['3cnt'], df['5cnt'], df['10cnt'], df['30cnt']):
    stan.append(int(df_stan))
    cnt3.append(int(df_cnt3))
    cnt5.append(int(df_cnt5))
    cnt10.append(int(df_cnt10))
    cnt30.append(int(df_cnt30))

In [151]:
def GetNumber(max_val, f1, f2, f3, f4):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = ("""
             select sum(case when row_num = 1 then num end) as n1
                  , sum(case when row_num = 2 then num end) as n2
                  , sum(case when row_num = 3 then num end) as n3
                  , sum(case when row_num = 4 then num end) as n4
                  , sum(case when row_num = 5 then num end) as n5
                  , sum(case when row_num = 6 then num end) as n6
                  , (select 3cnt  from in_list_2 where stan = {0}) as cnt3
                  , (select 5cnt  from in_list_2 where stan = {0}) as cnt5
                  , (select 10cnt from in_list_2 where stan = {0}) as cnt10
                  , (select 30cnt from in_list_2 where stan = {0}) as cnt30
               from (
             select num
                  , @rownum:=@rownum+1 as row_num
               from (
                    select seq, num   
                      from (select 3 as seq, num
                              from in_list 
                             where seq = {0} 
                               and val = 3 
                             order by rand() 
                             limit {1}
                           ) as t3
                    union all
                    select seq, num   
                      from (select 5 as seq, num
                              from in_list 
                             where seq = 974 
                               and val = 5 
                             order by rand() 
                             limit {2}
                           ) as t5 
                    union all
                    select seq, num   
                      from (select 10 as seq, num
                              from in_list 
                             where seq = 974 
                               and val = 10
                             order by rand() 
                             limit {3}
                           ) as t10
                    union all
                    select seq, num   
                    from (select 30 as seq, num
                            from in_list 
                           where seq = 974 
                             and val = 30 
                           order by rand() 
                           limit {4}
                         ) as t30
                     order by num 
                 ) as a
                 , (SELECT @rownum:=0) TMP
                  ) as b 
             ;
           """.format(max_val, f1, f2, f3, f4)
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [ ]:
data = GetNumber(stan[1], cnt3[1], cnt5[1], cnt10[1], cnt30[1])

for i in range(2, 16):
    data = data.append(GetNumber(stan[i] , cnt3[i], cnt5[i], cnt10[i], cnt30[i]), ignore_index = True)

data